# menu

Press run all.

Then you can choose what you want by inputting the desired number(s) between the brackets below and a list with all your requested information will be returned. (also remove the #)

    Input 1 for: Tweets sent by KLM
    Input 2 for: Tweets sent by British Airways
    Input 3 for: All tweets containing an @ someone
    Input 4 for: All replies sent by KLM
    Input 5 for: All replies sent by British Airways
    Input 6 for: All conversations between users and KLM (sorted on conversationnr)
    Input 7 for: All conversations between users and British Airways (sorted on conversationnr)
    Input 8 for: Sentiment analysis on KLM (needs polyglot)
    Input 9 for: Sentiment analysis on British Airways (needs polyglot)

In [ ]:
 execute([])


In [ ]:
class Filter:
    
    def __init__(self,dataframe,klmlist):
        self.data = dataframe
        try:
            self.klm = KLM(klmlist).KLMlist()
        except:
            self.klm = klmlist
        
    #gets tweets from Klm accounts
    def getKLM(self):
        get = self.data[self.data['user.id'].isin(self.klm)]
        return get
    #gets tweets from 
    def getBAW(self):
        get = self.data[self.data['user.id']==18332190]
        return get
    
    
    def getat(self):
        beer = self.data[self.data['text'].str.extract(r'.*(@[a-zA-Z0-9_]*)*.*',expand=False)!='']
        beer['uur'] = self.data['text'].str.extract(r'.*(@[a-zA-Z0-9_]*).*',expand=False)
        beer = beer[beer['uur']==beer['uur']]
        beer = beer.drop(['uur'],axis=1)
        return beer
    
    
    def tweetsKLM(self):
        klm_tweets = Filter(self.data,self.klm).getKLM()
        klm_tweets['in_reply_to_user_id_str'] = klm_tweets['in_reply_to_user_id_str'].astype(str)
        klm_replies = klm_tweets[klm_tweets['in_reply_to_user_id_str'] != 'nan']
        return klm_replies
    
    def tweetsBAW(self):
        klm_tweets = Filter(self.data,self.klm).getBAW()
        klm_tweets['in_reply_to_user_id_str'] = klm_tweets['in_reply_to_user_id_str'].astype(str)
        klm_replies = klm_tweets[klm_tweets['in_reply_to_user_id_str'] != 'nan']
        return klm_replies

    
    def conversattionneerKLM(self):
        #gets list of conversation starting points from filter
        Convlist = Filter(self.data,self.klm).tweetsKLM()
        #gets tweets which could be in the conversation
        Orgtweet = Filter(self.data,self.klm).getat()

        #gets list of users which talked to klm at some point
        Convlist['convpoint'] = Convlist['in_reply_to_user_id']
        #gets tweets from these users
        TweetsOforg = Orgtweet[Orgtweet['user.id'].isin(list(Convlist['convpoint']))]

        #Gets only the tweets which are @ any klm account
        TweetsOforg['twee'] = TweetsOforg['text'].str.extract(r'.*@((?:KLM|klm)(?:[a-zA-Z0-9_]*)).*',expand=False)
        TweetsOforg = TweetsOforg[TweetsOforg['twee'] == TweetsOforg['twee']]

        #Adds the convpoint column to be a common column with that of the conversation starting points
        TweetsOforg['convpoint'] = TweetsOforg['user.id']

        #merges dataframes
        tweets = pd.concat([Convlist,TweetsOforg]).sort_values('convpoint')

        #Copied from stackoverflow
        #gets an index for approx each conversation
        tweets.sort_values(['convpoint'], inplace=True)
        duplicated = tweets.duplicated(subset=['convpoint'], keep='first')
        new_group = ~duplicated
        tweets['conversationnr'] = new_group.cumsum()


        #drops unnecesairy axes
        tweets = tweets.drop(['convpoint','twee'],axis=1)

        return tweets
  

    def conversattionneerBAW(self):
        #gets list of conversation starting points from filter
        Convlist = Filter(self.data,self.klm).tweetsBAW()
        #gets tweets which could be in the conversation
        Orgtweet = Filter(self.data,self.klm).getat()

        #gets list of users which talked to klm at some point
        Convlist['convpoint'] = Convlist['in_reply_to_user_id']
        #gets tweets from these users
        TweetsOforg = Orgtweet[Orgtweet['user.id'].isin(list(Convlist['convpoint']))]

        #Gets only the tweets which are @ any klm account
        TweetsOforg['twee'] = TweetsOforg['text'].str.extract(r'.*@(British_Airways).*',expand=False)
        TweetsOforg = TweetsOforg[TweetsOforg['twee'] == TweetsOforg['twee']]

        #Adds the convpoint column to be a common column with that of the conversation starting points
        TweetsOforg['convpoint'] = TweetsOforg['user.id']

        #merges dataframes
        tweets = pd.concat([Convlist,TweetsOforg]).sort_values('convpoint')

        #Copied from stackoverflow
        #gets an index for approx each conversation
        tweets.sort_values(['convpoint'], inplace=True)
        duplicated = tweets.duplicated(subset=['convpoint'], keep='first')
        new_group = ~duplicated
        tweets['conversationnr'] = new_group.cumsum()


        #drops unnecesairy axes
        tweets = tweets.drop(['convpoint','twee'],axis=1)

        return tweets
    
    def toKLM(self):
        atSB = Filter(self.data,self.klm).getat()
        #Gets only the tweets which are @ any klm account
        atSB['twee'] = atSB['text'].str.extract(r'.*@((?:KLM|klm)(?:[a-zA-Z0-9_]*)).*',expand=False)        
        atKLM = atSB[atSB['twee'] == atSB['twee']]
        
        atKLM_en = atKLM[atKLM['lang']=='en']
        atKLM_nl = atKLM[atKLM['lang']=='nl']
        print('starting sentiment analysis')
        from polyglot.downloader import downloader
        downloader.download('sentiment2.nl')
        downloader.download('sentiment2.en')
        downloader.download('embeddings2.nl')
        downloader.download('embeddings2.en')
        downloader.download('ner2.en')
        downloader.download('ner2.nl')
        atKLM_en['sentiment'] = np.array([Filter(self.data,self.klm).polyglotPol(tweetText,'en') for tweetText in atKLM_en['text']])
        atKLM_nl['sentiment'] = np.array([Filter(self.data,self.klm).polyglotPol(tweetText,'nl') for tweetText in atKLM_nl['text']])
        return atKLM_en , atKLM_nl
        
    def clean_tweet(self,tweetText):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ",tweetText).split())

    def polyglotPol(self,tweet,lang):
        tweet = Filter(self.data,self.klm).clean_tweet(tweet)
        tek = Text(tweet)
        tek.language = lang
        return tek.polarity

        
    def toBAW(self):    
        atSB = Filter(self.data,self.klm).getat()
        #Gets only the tweets which are @ any klm account
        atSB['twee'] = atSB['text'].str.extract(r'.*@(British_Airways).*',expand=False)        
        atBaw = atSB[atSB['twee'] == atSB['twee']]
        
        atBaw_en = atBaw[atBaw['lang']=='en']
        from polyglot.downloader import downloader
        downloader.download('sentiment2.en')
        downloader.download('embeddings2.en')
        downloader.download('ner2.en')
        atBaw_en['sentiment'] = np.array([Filter(self.data,self.klm).polyglotPol(tweetText,'en') for tweetText in atBaw_en['text']])
        return atBaw_en
        
class KLM:
    
    def __init__(self,klmlist):
        self.klm = klmlist
    
    def KLMlist(self):
        klmNames = (open(self.klm,'r')).read()
        klmNameList = klmNames.split('\n')
        nameList = []
        for item in klmNameList:
            nameList.append(int(float(item)))
        return nameList    

In [ ]:
def execute(which)->list:
    retlist = []
    if 1 in which:
        retlist.append(Filter(Datafile,klmlist).getKLM())
    if 2 in which:
        retlist.append(Filter(Datafile,klmlist).getBAW())
    if 3 in which:
        retlist.append(Filter(Datafile,klmlist).getat())
    if 4 in which:
        retlist.append(Filter(Datafile,klmlist).tweetsKLM())
    if 5 in which:
        retlist.append(Filter(Datafile,klmlist).tweetsBAW())
    if 6 in which:
        retlist.append(Filter(Datafile,klmlist).conversattionneerKLM())
    if 7 in which:
        retlist.append(Filter(Datafile,klmlist).conversattionneerBAW())
    if 8 in which:
        retlist.append(Filter(Datafile,klmlist).toKLM())
    if 9 in which:
        retlist.append(Filter(Datafile,klmlist).toBAW())
    if 'dev_clean' in which:
        retlist.append(Filter(Datafile,klmlist).clean_tweet(input()))
    if 'dev_polyglot' in which:
        retlist.append(Filter(Datafile,klmlist).polyglotPol(input(),input()))
    return retlist

In [ ]:
import pandas as pd
import numpy as np
import re
try:
    from polyglot.text import Text
except:
    print('polyglot functions unavailable')
Datafile = pd.read_csv('files/DC_complete.airlines_1.csv')
klmlist = 'files/KLMnr.txt'